In [1]:
import json
import pandas as pd
import numpy as np

def json_to_dataframe(file):

    f = open ( file , "r") 
    data = json.loads(f.read())               #loading the json file.
    iid = []                                  
    tit = []                                  #Creating empty lists to store values.
    con = []
    Que = []
    Ans_st = []
    Txt = []
    
    for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
        
        title = data['data'][i]['title']
        for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
            
            context = data['data'][i]['paragraphs'][p]['context']
            for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
                
                question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
                Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
                for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags. 
                    
                    ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
                    text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']
                    
                    tit.append(title)
                    con.append(context)
                    Que.append(question)                    # Appending values to lists
                    iid.append(Id)
                    Ans_st.append(ans_start)
                    Txt.append(text)

    print('Done')      # for indication perpose.
    new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
    new_df.Id = iid
    new_df.title = tit           #intializing list values to the DataFrame.
    new_df.context = con
    new_df.question = Que
    new_df.ans_start = Ans_st
    new_df.text = Txt
    print('Done')      # for indication perpose.
    final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
    return final_df

In [2]:
train_data = json_to_dataframe('/kaggle/input/stanford-question-answering-dataset/train-v1.1.json')
train_data.head()

Done
Done


,Id,title,context,question,ans_start,text
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [3]:
import spacy
spacy_eng = spacy.load('en_core_web_sm')

class Vocabulary:
    PAD_token = 0   # Used for padding short sentences
    SOS_token = 1   # Start-of-sentence token
    EOS_token = 2   # End-of-sentence token

    def __init__(self, name):
        self.name = name
        self.word2index = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2}
        self.word2count = {}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>"}
        self.num_words = 3
        self.num_sentences = 0 
        self.longest_sentence = 0
    


    def add_word(self, word):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            
    def tokenizer(self,text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]
    
    def add_sentence(self, sentence):
        sentence_len = 0
        #for word in sentence.split(' '):
        for word in self.tokenizer(sentence):
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

In [4]:
# creating the vocabulary 
vocab = Vocabulary('qa')
vocab_questions = Vocabulary('q')

# adding words to the vocabulary 
for sentence in train_data['context']:
    vocab.add_sentence(sentence)

for sentence in train_data['question']:
    vocab.add_sentence(sentence)
    vocab_questions.add_sentence(sentence)

In [5]:
vocab.num_words

91507

In [6]:
sent_idxs = [1]
questions = []
contexts = []
for idx, sentence in enumerate (train_data['question']):
    i = 0
    
    for word in vocab.tokenizer(sentence):
        sent_idxs.append(vocab.to_index(word))
        i+=1
    while i < ((vocab_questions.longest_sentence) + 2):
        sent_idxs.append(0)
        i+=1
    questions.append(sent_idxs)
    sent_idxs = [1]

# converting list of word tokens to numpy array
import numpy as np
questions = np.array(questions)
questions.shape

(87599, 63)

In [7]:
contexts = []
sent_idxs = [1]
for sentence in train_data['context']:
    i = 0
    for word in vocab.tokenizer(sentence):
        sent_idxs.append(vocab.to_index(word))
        i+=1
    while i < ((vocab.longest_sentence) + 1):
        sent_idxs.append(0)
        i+=1
    contexts.append(sent_idxs)
    sent_idxs = [1]

# converting list of word tokens to numpy array
import numpy as np
contexts = np.array(contexts)
contexts.shape

(87599, 811)

In [8]:
for i in range(len(questions)):
    for j in range(1,vocab_questions.longest_sentence + 1):
        if questions[i,j] == 0:
            questions[i,j] = 2
            break
for i in range(len(contexts)):
    for j in range(1,vocab.longest_sentence + 1):
        if contexts[i,j] == 0:
            contexts[i,j] = 2
            break

In [9]:
vocabs = vocab.word2index.keys()

def load_embeds(root_dir):
    embeddings_index = dict()
    f = open(root_dir)

    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

    f.close()
    return embeddings_index
embeddings_index = load_embeds('../input/glove6b300dtxt/glove.6B.300d.txt')

In [10]:
embeddings_index['the']

array([ 4.6560e-02,  2.1318e-01, -7.4364e-03, -4.5854e-01, -3.5639e-02,
        2.3643e-01, -2.8836e-01,  2.1521e-01, -1.3486e-01, -1.6413e+00,
       -2.6091e-01,  3.2434e-02,  5.6621e-02, -4.3296e-02, -2.1672e-02,
        2.2476e-01, -7.5129e-02, -6.7018e-02, -1.4247e-01,  3.8825e-02,
       -1.8951e-01,  2.9977e-01,  3.9305e-01,  1.7887e-01, -1.7343e-01,
       -2.1178e-01,  2.3617e-01, -6.3681e-02, -4.2318e-01, -1.1661e-01,
        9.3754e-02,  1.7296e-01, -3.3073e-01,  4.9112e-01, -6.8995e-01,
       -9.2462e-02,  2.4742e-01, -1.7991e-01,  9.7908e-02,  8.3118e-02,
        1.5299e-01, -2.7276e-01, -3.8934e-02,  5.4453e-01,  5.3737e-01,
        2.9105e-01, -7.3514e-03,  4.7880e-02, -4.0760e-01, -2.6759e-02,
        1.7919e-01,  1.0977e-02, -1.0963e-01, -2.6395e-01,  7.3990e-02,
        2.6236e-01, -1.5080e-01,  3.4623e-01,  2.5758e-01,  1.1971e-01,
       -3.7135e-02, -7.1593e-02,  4.3898e-01, -4.0764e-02,  1.6425e-02,
       -4.4640e-01,  1.7197e-01,  4.6246e-02,  5.8639e-02,  4.14

In [11]:
import torch
def load_embed_weights(embeddings_index, embed_dim, vocab, vocab_size):
    matrix_len = vocab_size
    weights_matrix = np.zeros((matrix_len, embed_dim))
    words_found = 0

    for i, word in enumerate(vocab):
        try: 
            weights_matrix[i] = embeddings_index[word]
            words_found += 1
        except KeyError:
            weights_matrix[i] = np.random.normal(scale=0.6, size=(embed_dim, ))
    weights_matrix = torch.tensor(weights_matrix)
    return weights_matrix
weights_matrix = load_embed_weights(embeddings_index, 300, vocabs, vocab.num_words)
weights_matrix.shape

torch.Size([91507, 300])

In [12]:
train_data.head()

,Id,title,context,question,ans_start,text
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [13]:
start_word_idx = []
for i in range(len(train_data)):
    context = train_data['context'].iloc[i]
    text = train_data['text'].iloc[i]
    start = train_data['ans_start'].iloc[i]
    idx = len(vocab.tokenizer(text)[0])
    context = context[:(start+idx)]
    context = vocab.tokenizer(context)
    start_word_idx.append(len(context)-1)
train_data['start_word'] = start_word_idx

In [14]:
train_data['end_word'] = [(train_data['start_word'].iloc[i]+ len(vocab.tokenizer(train_data['text'].iloc[i])) - 1) for i in range(len(train_data))]

In [15]:
train_data.head()

,Id,title,context,question,ans_start,text,start_word,end_word
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous,102,104
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ,37,41
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building,57,59
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection,76,82
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary,17,23


In [16]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from glob import glob
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class SquadDataset(Dataset):
    
    def __init__(self, train_data,contexts, questions):
        self.contexts = contexts
        self.questions = questions 
        self.train_data = train_data
    
    def __getitem__(self, index):
        return self.contexts[index], self.questions[index], self.train_data['start_word'].iloc[index]+1, self.train_data['end_word'].iloc[index]+1
    def __len__(self):
        return len(self.questions)


data = SquadDataset(train_data, contexts, questions)


In [17]:
batch_size = 32
data_len = len(data)
trainset, valset = random_split(data, [int(0.8*data_len), (data_len - int(0.8*data_len))])
trainloader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle = True)
valloader = DataLoader(dataset=valset, batch_size=batch_size, shuffle = True)


In [18]:
for c,q,s,e in trainloader:
    for i in q[0]:
        print(vocab.to_word(int(i)), end = ' ')
    print()
    print()
    for i in c[0]:
        print(vocab.to_word(int(i)), end = ' ')
    break

<SOS> who was the commissioner of the arena football league for the first half of 2008 ? <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> 

<SOS> after 12 years as commissioner of the afl , david baker retired unexpectedly on july 25 , 2008 , just two days before arenabowl xxii ; deputy commissioner ed policy was named interim commissioner until baker 's replacement was found . baker explained , " when i took over as commissioner , i thought it would be for one year . it turned into 12 . but now it 's time . " <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

In [19]:
class Model_1(nn.Module):
    
    def __init__(self, embed_size, hidden_size, context_size, vocab_size ):
        super().__init__()
        
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.context_size = context_size
        
        self.embed = nn.Embedding(num_embeddings = self.vocab_size, embedding_dim = self.embed_size)
        self.embed.weight.requires_grad = False
        self.embed.load_state_dict({'weight': weights_matrix})

        
      
        self.question_lstm = nn.LSTM(input_size = self.embed_size, hidden_size = self.hidden_size, bidirectional = True, batch_first = True)
        self.context_lstm = nn.LSTM(input_size = self.embed_size, hidden_size = self.hidden_size, bidirectional = True, batch_first = True)
        
        self.attention = nn.Linear(2*self.hidden_size, 2*self.hidden_size, bias = False)

        self.start_fc = nn.Linear(self.context_size, self.context_size)
        self.end_fc = nn.Linear(self.context_size, self.context_size)
        
        

        

    def forward(self, contexts, questions):
        
        contexts_embed = self.embed(contexts)
        questions_embed = self.embed(questions)
       
        questions_rep, _ = self.question_lstm(questions_embed)
        contexts_rep, _ = self.context_lstm(contexts_embed)                     #[64,811, 400]
        
        questions_rep = questions_rep[:,-1,:].unsqueeze(1)                      #[64,1, 400]
        a = []
        for i in range(contexts_rep.shape[1]):
            c = self.attention(contexts_rep[:,i,:]).unsqueeze(2)                # [64, 400, 1]
        
            c2 = []
            for batch in range(contexts_rep.shape[0]):
                c1 = questions_rep[batch,:,:] @ c[batch,:,:]  # [1,1]
                c2.append(c1)
            c2 = torch.stack(c2)
            a.append(c2)                              #[64,1,1]
        
        a = torch.stack(a)
        a = a.permute(1,0,2,3).squeeze(-1).squeeze(-1)

        a = F.softmax(a, dim = 1)    # [64,811]
        start_probs = self.start_fc(a)    #[64,811]
        end_probs = self.end_fc(a)        #[64,811]
        
        
        
        return start_probs, end_probs
        

In [20]:
class Model_2(nn.Module):
    
    def __init__(self, embed_size, hidden_size, context_size, vocab_size ):
        super().__init__()
        
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.context_size = context_size
        
        self.embed = nn.Embedding(num_embeddings = self.vocab_size, embedding_dim = self.embed_size)
        self.embed.weight.requires_grad = False
        self.embed.load_state_dict({'weight': weights_matrix})

        self.lstm = nn.LSTM(input_size = self.embed_size, hidden_size = self.hidden_size, bidirectional = True, batch_first = True)
        
        self.q_fc = nn.Linear(2*self.hidden_size, 2*self.hidden_size)
        self.c_fc = nn.Linear(4*self.hidden_size, 2*self.hidden_size)
        
        self.start_fc = nn.Linear(2*self.hidden_size, 1) 
        
        self.end_lstm = nn.LSTM(input_size = 2*self.hidden_size, hidden_size = self.hidden_size, bidirectional = True, batch_first = True)
        self.end_fc = nn.Linear(2*self.hidden_size, 1)
        
        self.tanh = nn.Tanh()

    def forward(self, contexts, questions):
        
        contexts_embed = self.embed(contexts)
        questions_embed = self.embed(questions)

        questions_rep, _ = self.lstm(questions_embed)                   # [64,63,400]
        contexts_rep, _ = self.lstm(contexts_embed)                     #[64,811, 400]
        
        questions_rep = self.tanh(self.q_fc(questions_rep))
        
        A = []
        for i in range(questions_rep.shape[0]):
            A.append(questions_rep[i,:,:] @ contexts_rep[i,:,:].T)
        A = torch.stack(A)                                           # [64,63,811]  
        A = F.softmax(A, dim = 2)
        
        Q = []
        for i in range(questions_rep.shape[0]):
            Q.append(A[i,:,:].T @ questions_rep[i,:,:])  
        Q = torch.stack(Q)                                          # [64,811,400]
        
        M = torch.cat((Q, contexts_rep), dim = -1)
                   
        M = self.tanh(self.c_fc(M))   # [64,811,400] 
        
        start_probs = self.start_fc(M).squeeze(-1)
        
        Hf, _ = self.end_lstm(M)                                  # [64,811,400]

        end_probs = self.end_fc(Hf).squeeze(-1)
        
        
        return start_probs, end_probs

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embed_size = 300
hidden_size = 200
context_size = contexts.shape[1]
vocab_size = vocab.num_words

model1 = Model_1(embed_size, hidden_size, context_size, vocab_size )
model1 = model1.to(device)

model2 = Model_2(embed_size, hidden_size, context_size, vocab_size )
model2 = model2.to(device)

criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer1 = torch.optim.Adam(model1.parameters(), lr = 1e-2)
optimizer2 = torch.optim.Adam(model2.parameters(), lr = 1e-2)


In [22]:
device

device(type='cuda')

In [ ]:
import pyprind
best_loss = 100
def train(trainloader, model1, model2, criterion, optimizer1, optimizer2):
    train_loss = []
    bar = pyprind.ProgBar(len(trainloader), bar_char='█')
    for batch_idx, (c,q,s,e) in enumerate(trainloader):
        c = c.to(device)
        q = q.to(device)
        s = s.to(device)
        e = e.to(device)
        start_probs1, end_probs1 = model1(c,q)
        start_probs2, end_probs2 = model2(c,q)
        start_probs = start_probs1 + start_probs2
        end_probs = start_probs1 + end_probs2
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        loss1 = criterion(start_probs, s)
        loss2 = criterion(end_probs, e)
        loss = loss1 + loss2
        train_loss.append(loss.item())
        loss.backward()
        optimizer1.step()
        optimizer2.step()
        
        bar.update()
        
        if (batch_idx%5 == 0): 
            print('batch({}/{})  train loss : {}\n'.format(batch_idx, len(trainloader),np.mean(train_loss)))
    
    
    
    torch.save(model1.state_dict(), 'weights_model_1.pth')
    torch.save(model2.state_dict(), 'weights_model_2.pth')
    best_loss = np.mean(train_loss)
    

In [ ]:
epochs = 1
#model1.load_state_dict(torch.load('../input/qna-weights/weights_model_1.pth'))
#model2.load_state_dict(torch.load('../input/qna-weights/weights_model_2.pth'))
for epoch in range(epochs):
    train(trainloader, model1, model2, criterion, optimizer1, optimizer2)

In [23]:
model1.load_state_dict(torch.load('../input/qna-weights1/weights_model_1.pth'))
model2.load_state_dict(torch.load('../input/qna-weights1/weights_model_2.pth'))
for batch_idx, (c,q,s,e) in enumerate(trainloader):
    c = c.to(device)
    q = q.to(device)
    s = s.to(device)
    e = e.to(device)
    print(s[1])
    start_probs1, end_probs1 = model1(c,q)
    start_probs2, end_probs2 = model2(c,q)
    start_probs = start_probs1 + start_probs2
    end_probs = start_probs1 + end_probs2
    break

tensor(85, device='cuda:0')


# Checking for a random question

## question and  predicted answer

In [77]:
t = 3
print('question: ', end = ' ')
for i in q[t]:
    if int(i) == 0:
        break
    if i!= 1 and i!=2:
        print(vocab.to_word(int(i)), end = ' ')
print()
print()
print('actual answer: ', end = ' ')
for i in range(int(s[t]),(int(e[t])+1)):
    print(vocab.to_word(int(c[t][i])), end = ' ')
s1 = torch.argmax(start_probs, dim = 1)
e1 = torch.argmax(end_probs, dim = 1)
print()
print()
print('predicted answer: ', end = ' ')
for i in range(int(s1[t]),(int(e1[t])+1)):
    print(vocab.to_word(int(c[t][i])), end = ' ')



question:  who is he current minority leader ? 

actual answer:  nancy pelosi 

predicted answer:  nancy pelosi 